# I. $LU$ factorization of a square matrix
When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} m = (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
A = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        A[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(A)

6

In [3]:
np.round(A,3)

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.875, 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

In [ ]:
L, U = diy_lu(A)

print(np.round(L,3), "\n")
print(np.round(U,3), "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(np.round(L@U - A,3))

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.630e-01  4.570e-01  5.970e-01  7.010e-01]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.200e-02 -4.500e-02 -6.500e-02]
 [ 0.000e+00 -0.000e+00  0.000e+00  0.000e+00  1.000e-03  2.000e-03]
 [ 0.000e+00  0.000e+00  0.000e+00 -0.000e+00  0.000e+00 -0.000e+00]] 

[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0. -0.]
 [ 0.  0.  0. -0. -0.  0.]
 [ 0.  0.  0. -0. -0.  0.]
 [ 0.  0. -0. -0.  0.  0.]]


# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [4]:
A1 = A.copy()
A1[1, 1] = 3

In [5]:
np.round(A1,3)

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 3.   , 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

In [6]:
np.linalg.matrix_rank(A1)

6

In [7]:
L, U= diy_lu(A1)

print(L, U)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


<ipython-input-1-8918e3eddeb9>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-1-8918e3eddeb9>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-1-8918e3eddeb9>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-1-8918e3eddeb9>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


The LU decomposition from scipy.linalg.lu already implements pivoting other sophisticated controls 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html

$$A = P L U$$

```python

 P ,  L,  U  = scipy.linalg.lu(a, permute_l=False, overwrite_a=False, check_finite=True)

# Returns
# (If permute_l == False)
# P : Permutation matrix
# L : Lower triangular or trapezoidal matrix with unit diagonal. K = min
# U : Upper triangular or trapezoidal matrix
```

In [8]:
from scipy import linalg
P ,  L,  U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",np.round(U,3), "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")


P
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

L
 [[1.         0.         0.         0.         0.         0.        ]
 [1.         1.         0.         0.         0.         0.        ]
 [1.         0.5        1.         0.         0.         0.        ]
 [1.         0.72727273 0.70588235 1.         0.         0.        ]
 [1.         0.85714286 0.40993789 0.83482143 1.         0.        ]
 [1.         0.94117647 0.17849899 0.4255677  0.78870221 1.        ]] 

U
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.510e-01 -5.790e-01 -7.330e-01 -8.440e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  2.400e-02  4.900e-02  7.000e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.000e-03 -2.000e-03]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]] 

A= P@L@U
 [[3.         3.   

In [ ]:
P ,  L,  U = linalg.lu(A1)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",np.round(U,3), "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")


P
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.        ]
 [ 1.         -0.          1.          0.          0.          0.        ]
 [ 1.          0.72727273  0.1512605   1.          0.          0.        ]
 [ 1.          0.85714286  0.08784383  0.51421669  1.          0.        ]
 [ 1.          0.94117647  0.03824978  0.2076544   0.64143198  1.        ]] 

U
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -9.200e-02 -1.480e-01 -1.860e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  2.000e-03  4.000e-03]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -0.000e

### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [10]:
def leading_minors_test(A):
   """
    Function to check all leading Minors of a Matrix not equal to 0
   """
   print("Check matrix A =\n",A)
   ret = True
   for i in range(1,A.shape[0]+1):   
     sub_matrix = A[:i,:i]
     leading_minor = np.linalg.det(sub_matrix)
     if( leading_minor == 0 ): ret = False
     print(i,"th leading minor of A =" , leading_minor)  
   print("determinant of A =",np.linalg.det(A))          
   return ret

leading_minors_test(A), leading_minors_test(A1)

Check matrix A =
 [[3.         3.         3.         3.         3.         3.        ]
 [3.         1.875      1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.65217391 0.46875    0.36585366 0.3       ]
 [3.         0.88235294 0.51724138 0.36585366 0.28301887 0.23076923]
 [3.         0.75       0.42857143 0.3        0.23076923 0.1875    ]]
1 th leading minor of A = 3.0000000000000004
2 th leading minor of A = -3.375
3 th leading minor of A = -0.8859990277102598
4 th leading minor of A = 0.019467001032005795
5 th leading minor of A = 1.5728974209748546e-05
6 th leading minor of A = -2.4923994636580374e-10
determinant of A = -2.4923994636580374e-10
Check matrix A =
 [[3.         3.         3.         3.         3.         3.        ]
 [3.         3.         1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.652

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

##Case 1

In [11]:
def diy_lu_column_pivot(a):
  """Construct the LU decomposition of the input matrix.

  LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .
  """
  N = a.shape[0]
  u = a.copy()
  L = np.eye(N)

  P1 = np.eye(N) # Permutacion Inicial 1
  P2 = np.eye(N) # Permutacion Inicial 2

  for j in range(N-1):
    lam = np.eye(N)
    i = j
    while (u[i, i] == 0.0 and i < N-1):
      tmp = np.argmax(abs(u[i:,i])) + i
      P2[i], P2[tmp] = P2[tmp], P2[i].copy()
      u = P2 @ u
      i += 1
    gamma = u[j+1:, j] / u[j, j]
    lam[j+1:, j] = -gamma
    u = lam @ u

    lam[j+1:, j] = gamma
    L = L @ lam @ P1
  return  L, u, P

##Case 2

In [12]:
def diy_lu_column_pivot_reconstruct(a):
    """Construct the LU decomposition of the input matrix.
    
    LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .
    """
    N = a.shape[0]

    u = a.copy()
    L = np.eye(N)

    for j in range(1, N-1):
        lam = np.eye(N)
        gamma = u[j-1:, j] / u[j, j]
        lam[j-1:, j] = -gamma
        u = lam @ u

        lam[j-1:, j] = gamma
        L = L @ lam

    return  L, u

In [13]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u = diy_lu_column_pivot_reconstruct(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")

L
 [[1.00000000e+00 1.60000000e+00 1.99466667e+01 6.22819556e+02
  4.60469109e+04 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.24666667e+01 3.89262222e+02
  2.87793193e+04 0.00000000e+00]
 [0.00000000e+00 7.27272727e-01 1.00666667e+01 3.14324040e+02
  2.32389156e+04 0.00000000e+00]
 [0.00000000e+00 5.71428571e-01 8.28530021e+00 2.59702222e+02
  1.92005613e+04 0.00000000e+00]
 [0.00000000e+00 4.70588235e-01 7.00459770e+00 2.20242570e+02
  1.62841913e+04 0.00000000e+00]
 [0.00000000e+00 4.00000000e-01 6.05523810e+00 1.90836622e+02
  1.41110053e+04 1.00000000e+00]] 

u
 [[-1.80000000e+00 -1.66533454e-16  8.18181818e-01  1.28571429e+00
   1.58823529e+00  1.80000000e+00]
 [-1.32000000e+01 -1.87500000e+00 -4.24227174e-17  5.12422360e-01
   6.69371197e-01  7.07142857e-01]
 [-1.12909091e+01  0.00000000e+00  1.09382596e-01 -7.47622048e-19
  -6.98245504e-02 -1.07532468e-01]
 [-1.10012422e+01  0.00000000e+00  0.00000000e+00 -4.06885344e-03
   9.25278772e-20  4.53347894e-03]
 [-1.44263593e-01

In [14]:
a1 = a.copy()
a1[1, 1] = 3

L, u, = diy_lu_column_pivot_reconstruct(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a1, "\n")

L
 [[ 1.00000000e+00  1.00000000e+00 -5.19444444e+00 -4.81637902e+01
  -6.54898709e+03  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -5.19444444e+00 -4.81637902e+01
  -6.54898709e+03  0.00000000e+00]
 [ 0.00000000e+00  4.54545455e-01 -1.36111111e+00 -1.26204584e+01
  -1.71604475e+03  0.00000000e+00]
 [ 0.00000000e+00  3.57142857e-01 -1.22601794e+00 -1.03678511e+01
  -1.40975041e+03  0.00000000e+00]
 [ 0.00000000e+00  2.94117647e-01 -1.08524904e+00 -8.80727414e+00
  -1.19655369e+03  0.00000000e+00]
 [ 0.00000000e+00  2.50000000e-01 -9.64682540e-01 -7.65296785e+00
  -1.03894367e+03  1.00000000e+00]] 

u
 [[ 0.00000000e+00  0.00000000e+00  1.63636364e+00  1.92857143e+00
   2.11764706e+00  2.25000000e+00]
 [ 5.50000000e+00 -3.00000000e+00 -1.06116763e-16 -2.13509317e-01
  -2.78904665e-01 -2.94642857e-01]
 [-9.97268113e+00  0.00000000e+00 -2.62518230e-01 -1.06878155e-17
   9.11617987e-02  1.25681983e-01]
 [-3.69142686e+01  0.00000000e+00  0.00000000e+00  1.78125362e-02
  -2.11168434e-1

In [15]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, = diy_lu_column_pivot_reconstruct(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a2, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a2\n",L@u - a2, "\n")

L
 [[1.         0.42857143 0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.28571429 1.         0.        ]
 [0.         0.28571429 1.33333333 1.        ]] 

u
 [[ 1.85714286e+00  1.66533454e-16  1.00000000e+00]
 [-5.00000000e+00 -7.00000000e+00  0.00000000e+00]
 [-2.57142857e+00  0.00000000e+00 -3.00000000e+00]
 [ 3.14285714e+00  0.00000000e+00  0.00000000e+00]] 

L@u
 [[-0.28571429 -3.          1.        ]
 [-5.         -7.          0.        ]
 [-9.         -9.         -3.        ]
 [-1.71428571 -2.         -4.        ]] 

a2
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]] 

L@u - a2
 [[ -4.28571429  -6.           0.        ]
 [-10.         -14.           0.        ]
 [-18.         -18.          -6.        ]
 [ -9.71428571  -4.          -8.        ]] 



# 2. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and then premultply by lower triangular elemetary matrices $\Lambda_j$, $A$  it is transformed in an  upper triangular matrix $U$

$$  
(\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)A  = U
$$
and 
$$  
A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U
$$


So the $LU$ column pivot factorization of $A$ is 
$$
A = LU
$$
ith
$$  
U = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1) A
$$
$$
\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}
$$
because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$.

Note that 
$$
L= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
$$
is not exactly a lower tiangular matrix but a row permutated lower tiangular matrix.





$$PA = LU$$

In [20]:
def swap_rows_matrix( N , row_i , row_j ):
  """Returns the Permutation matrix of size N that swaps rows i and j

    """
  P_ij = np.eye(N)
  P_ij[[row_i,row_j]] = P_ij[[row_j,row_i]] 
  return P_ij

def max_element_in_column( matrix , column , row_range ):
  first_value = True
  for row in row_range:
    if first_value == True or matrix[row][column] > best:
      best = matrix[row][column]
      best_position = row
      first_value = False
  return best_position

In [36]:
def diy_lu_column_pivot_reconstruct(A):
    """Construct the LU decomposition of the input matrix.
    
    LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .
    """
   # ... ENTER YOUR CODE HERE ...
   
    
    U = A.copy()
    L = np.eye(N)
    for j in range(N-1):
      pivot = max_element_in_column( U , j , range(j,N) )
      p = swap_rows_matrix( N , j , pivot )
      
      U = p @ U

      lam = np.eye(N)
      gamma = U[j+1:, j] / U[j, j]
      lam[j+1:, j] = -gamma
      U = lam @ U

      lam[j+1:, j] = gamma
      L = L @ np.transpose(p)
      L = L @ lam
    P = np.eye(N); 

    return  P, L , U

In [35]:
A = np.array([[4,3,1], [5,7,0], [9,9,3]])

#P, L, U, = diy_lu_column_pivot_reconstruct(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
#print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

L
 [[1.         0.42857143 0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.28571429 1.         0.        ]
 [0.         0.28571429 1.33333333 1.        ]] 

U
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -2.25000000e+00 -2.57142857e+00 -2.70000000e+00
  -2.76923077e+00 -2.81250000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.50649351e-01 -5.78571429e-01
  -7.33031674e-01 -8.43750000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.42136380e-02
   4.86615163e-02  6.96142093e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -6.46187856e-04 -1.51605479e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  6.73010748e-06]] 

A
 [[4 3 1]
 [5 7 0]
 [9 9 3]] 



In [37]:
from scipy import linalg

A = np.array([[4,3,1], [5,7,0], [9,9,3]])

P,L,U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]] 

L
 [[ 1.          0.          0.        ]
 [ 0.55555556  1.          0.        ]
 [ 0.44444444 -0.5         1.        ]] 

U
 [[ 9.          9.          3.        ]
 [ 0.          2.         -1.66666667]
 [ 0.          0.         -1.16666667]] 

A= P@L@U
 [[4. 3. 1.]
 [5. 7. 0.]
 [9. 9. 3.]] 

A
 [[4 3 1]
 [5 7 0]
 [9 9 3]] 

L@u - A
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 



In [38]:
N = 6
A = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        A[i, j] = 3. / (0.6*i*j + 1)

P, L, U, = diy_lu_column_pivot_reconstruct(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")




P
 [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]] 

L
 [[1.         0.         0.         0.         0.         0.        ]
 [1.         1.         0.         0.         0.         0.        ]
 [1.         1.45454545 0.37433155 1.         0.         0.        ]
 [1.         1.71428571 0.65217391 0.83482143 1.26790567 1.        ]
 [1.         1.88235294 0.85192698 0.4255677  1.         0.        ]
 [1.         2.         1.         0.         0.         0.        ]] 

U
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -1.12500000e+00 -1.63636364e+00 -1.92857143e+00
  -2.11764706e+00 -2.25000000e+00]
 [ 0.00000000e+00  0.00000000e+00  7.01298701e-01  1.15714286e+00
   1.46606335e+00  1.68750000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.42136380e-02
   4.86615163e-02  6.96142093e-02]
 [ 0.00000000e+00  0.00000000e+00  0.0000

In [39]:
from scipy import linalg

P,L,U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

L
 [[1.         0.         0.         0.         0.         0.        ]
 [1.         1.         0.         0.         0.         0.        ]
 [1.         0.5        1.         0.         0.         0.        ]
 [1.         0.72727273 0.70588235 1.         0.         0.        ]
 [1.         0.85714286 0.40993789 0.83482143 1.         0.        ]
 [1.         0.94117647 0.17849899 0.4255677  0.78870221 1.        ]] 

U
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -2.25000000e+00 -2.57142857e+00 -2.70000000e+00
  -2.76923077e+00 -2.81250000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.50649351e-01 -5.78571429e-01
  -7.33031674e-01 -8.43750000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.42136380e-02
   4.86615163e-02  6.96142093e-02]
 [ 0.00000000e+00  0.00000000e+00  0.0000

In [40]:
A[1, 1] = 3

P, L, U, = diy_lu_column_pivot_reconstruct(A1)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]] 

L
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] 

U
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] 

A= P@L@U
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] 

A
 [[3.         3.         3.         3.         3.         3.        ]
 [3.         3.         1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.65217391 0.46875    0.36585366 0.3       ]
 [3.         0.88235294 0.51724138 0.36585366 0.28301887 0.23076923]
 [3.     

<ipython-input-36-ad74be00b6fd>:18: RuntimeWarning: divide by zero encountered in true_divide
  gamma = U[j+1:, j] / U[j, j]
<ipython-input-36-ad74be00b6fd>:20: RuntimeWarning: invalid value encountered in matmul
  U = lam @ U
<ipython-input-36-ad74be00b6fd>:24: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-36-ad74be00b6fd>:15: RuntimeWarning: invalid value encountered in matmul
  U = p @ U


In [41]:
from scipy import linalg

P,L,U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.        ]
 [ 1.         -0.          1.          0.          0.          0.        ]
 [ 1.          0.72727273  0.1512605   1.          0.          0.        ]
 [ 1.          0.85714286  0.08784383  0.51421669  1.          0.        ]
 [ 1.          0.94117647  0.03824978  0.2076544   0.64143198  1.        ]] 

U
 [[ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00
   3.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00 -2.25000000e+00 -2.57142857e+00 -2.70000000e+00
  -2.76923077e+00 -2.81250000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.63636364e+00 -1.92857143e+00
  -2.11764706e+00 -2.25000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.24730366e-02
  -1.48456245e-01 -1.85637892e-01]
 [ 0.